# Ejemplo 2 Configuración de un clúster HDFS con Docker Compose.

Como ingeniero de datos, siempre me ha fascinado el poder de los sistemas distribuidos. Recientemente, me embarqué en un viaje para configurar un clúster Hadoop Distributed File System (HDFS) utilizando Docker Compose. Este artículo comparte mi experiencia y proporciona una guía paso a paso para aquellos que buscan replicar esta configuración.

### 

- ¿Por qué Docker Compose para HDFS?

Docker Compose ofrece una forma cómoda de definir y ejecutar aplicaciones Docker multicontenedor. Para un clúster HDFS, que consta de varios nodos (NameNode y DataNodes), Docker Compose proporciona una solución ideal para crear un entorno reproducible y fácil de gestionar.

- Configuración

Nuestro cluster HDFS consistirá en un NameNode y dos DataNodes. Esto es lo que necesitarás:  

1. Docker y Docker Compose instalados en su sistema
2. Basic understanding of HDFS and Docker concepts

1- Estructura del proyecto

En primer lugar, cree una estructura de directorios para su proyecto:

In [ ]:
hdfs-docker-cluster/
│
├── docker-compose.yml
├── hadoop_config/
│   ├── core-site.xml
│   ├── hdfs-site.xml
│   └── ... (other Hadoop configuration files)
├── start-hdfs.sh
├── init-datanode.sh
├── hadoop_namenode/
├── hadoop_datanode1/
└── hadoop_datanode2/

In [ ]:
mkdir -p hdfs-docker-cluster/hadoop_config hdfs-docker-cluster/hadoop_namenode hdfs-docker-cluster/hadoop_datanode1 hdfs-docker-cluster/hadoop_datanode2

1.1 Navega al directorio principal del proyecto

In [ ]:
cd hdfs-docker-cluster

1.2 Crea los archivos necesarios dentro de hdfs-docker-cluster:

In [ ]:
touch docker-compose.yml start-hdfs.sh init-datanode.sh
touch hadoop_config/core-site.xml hadoop_config/hdfs-site.xml

2. Configuración del Docker Compose

Cree un archivo `docker-compose.yml` con el siguiente contenido:

In [ ]:
# yml
version: '3'

services:
  namenode:
    image: apache/hadoop:3.3.5
    container_name: namenode
    hostname: namenode
    user: root
    environment:
      - HADOOP_HOME=/opt/hadoop
    volumes:
      - ./hadoop_namenode:/opt/hadoop/data/nameNode
      - ./hadoop_config:/opt/hadoop/etc/hadoop
      - ./start-hdfs.sh:/start-hdfs.sh
    ports:
      - "9870:9870"
    command: [ "/bin/bash", "/start-hdfs.sh" ]
    networks:
      hdfs_network:
        ipv4_address: 172.20.0.2

  datanode1:
    image: apache/hadoop:3.3.5
    container_name: datanode1
    hostname: datanode1
    user: root
    environment:
      - HADOOP_HOME=/opt/hadoop
    volumes:
      - ./hadoop_datanode1:/opt/hadoop/data/dataNode
      - ./hadoop_config:/opt/hadoop/etc/hadoop
      - ./init-datanode.sh:/init-datanode.sh
    depends_on:
      - namenode
    command: [ "/bin/bash", "/init-datanode.sh" ]
    networks:
      hdfs_network:
        ipv4_address: 172.20.0.3

  datanode2:
    # Similar configuration to datanode1, with different container_name and IP

networks:
  hdfs_network:
    ipam:
      driver: default
      config:
        - subnet: 172.20.0.0/16

3- Inicializar scripts

Crear dos scripts para inicializar el NameNode y el DataNode:

`start-hdfs.sh` for the NameNode:

In [ ]:
#!/bin/bash
if [ ! -d "/opt/hadoop/data/nameNode/current" ]; then
    echo "Formatting NameNode..."
    hdfs namenode -format
fi
hdfs namenode

`init-datanode.sh` for the DataNodes:

In [ ]:
#!/bin/bash
rm -rf /opt/hadoop/data/dataNode/*
chown -R hadoop:hadoop /opt/hadoop/data/dataNode
chmod 755 /opt/hadoop/data/dataNode
hdfs datanode

4- Configuración de Hadoop

En el directorio `hadoop_config/`, coloque sus archivos de configuración de Hadoop. Los archivos clave son core-site.xml y hdfs-site.xml. Asegúrese de que están configurados correctamente para su clúster HDFS.

5- Lanzar el cluster

Una vez que los contenedores estén en funcionamiento, puede verificar la funcionalidad del clúster:  
1. Accede a la interfaz web de NameNode en `http://localhost:9870`  
2. Use HDFS commands through the NameNode container: `docker exec -it namenode hdfs dfs -ls /`


In [ ]:
docker exec -it namenode hdfs dfs -ls /

Añade un archivo a HDFS:

In [ ]:
echo "Hello, HDFS" > test.txt
docker cp test.txt namenode:/tmp/
docker exec -it namenode hdfs dfs -put /tmp/test.txt /

Ver el archivo en HDFS:

In [ ]:
docker exec -it namenode hdfs dfs -cat /test.txt

También puede consultar el estado del clúster en la interfaz web.

![1](1.png)

Aquí podemos ver nuestros dos datanodes, y también el histograma de uso de los procesos que hemos probado.  
Incluso podemos echar un vistazo a los archivos que hemos creado y movido en el directorio browse del servidor web:

![2](2.png)